In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
#import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/Desktop/PHD2022/Autumn/CSE 579/CS_project/LQ_Game_Solver/CS_project/Project.toml`


In [2]:
using LinearAlgebra
using Plots
using SparseArrays
using ForwardDiff
#using ControlSystems

### 2D Point Mass Continous dynamics

In [3]:
c = 0.0
m = 1.0
function point_mass(x, u)
   # x = state[1]                #x_1 = ẋ
    ẋ = x[3]
    ẍ = -(c/m)*ẋ + u[1]/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m 
    #y = state[3]
    ẏ = x[4]
    ÿ = -(c/m)*ẏ + u[2]/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m 
    return [ẋ; ẏ; ẍ; ÿ]
end

point_mass (generic function with 1 method)

### Linearizing and discretizing the dynamics

In [4]:
function lin_dyn_discrete(dynamics, x, u, dt)
    A = ForwardDiff.jacobian(dx -> dynamics(dx, u), x)
    B = ForwardDiff.jacobian(du -> dynamics(x, du), u)
    A = dt .* A + I
    B = dt .* B
    return A, B
end
x = [0; 0; 0 ;0]
u = [0; 0]
A, B = lin_dyn_discrete(point_mass, x, u, 0.01)

4×2 Matrix{Float64}:
 0.0   0.0
 0.0   0.0
 0.01  0.0
 0.0   0.01

### Cost Function (add distance constrain to arbitary static object)

In [43]:
Q = I(4) #state cost for point mass 1
R = I(2)
function cost(x, u)
    c = 0.5*x'*Q*x + 0.5*u'*R*u #+ max(0, (x[1] - x[3])^2)
    return c
end

cost (generic function with 1 method)

### Evaluating Cost function

In [44]:
x = [0; 0; 1; 0]
u = [1; 3]
cost(x, u)

5.5

### Quadritizing cost function

In [46]:
function quadratic_cost(cost_func, x, u)
    """
    2nd order Taylor expansion of cost at t
    I neglected the mixed paritals in the hessian
    """
    Q = ForwardDiff.hessian(dx -> cost_func(dx, u), x)
    l = ForwardDiff.gradient(dx -> cost_func(dx, u), x)
    R = ForwardDiff.hessian(du -> cost_func(x, du), u)
    r = ForwardDiff.gradient(du -> cost_func(x, du), u)
    cost = 0.5 * x' * (Q*x + 2*l) + 0.5 * u' * (R*u + 2*r)
    return cost, Q, l, R, r
end

quadratic_cost (generic function with 1 method)

#### Point Mass Discrete

$$ \frac{d}{dt}x = Ax + \sum Bu$$

Single 2D point mass:
$$\frac{d}{dt}\begin{bmatrix} x \\ y \\ \dot{x} \\ \dot{y}\end{bmatrix} =  
 \begin{bmatrix} 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & -c/m & 0 \\ 0 & 0 & 0 & -c/m \end{bmatrix}
 \begin{bmatrix} x \\ y \\ \dot{x} \\ \dot{y}\end{bmatrix} + 
 \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 1/m & 0 \\ 0 & 1/m \end{bmatrix}
 \begin{bmatrix} u_x \\ u_y\end{bmatrix} $$

two 2D point masses:
$$\frac{d}{dt}\begin{bmatrix} x_1 \\ y_1 \\ \dot{x}_1 \\ \dot{y}_1 \\ x_2 \\ y_2 \\ \dot{x}_2 \\ \dot{y}_2\end{bmatrix} =  
 \begin{bmatrix} 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\ 
                0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\ 
                0 & 0 & -c/m_1 & 0 & 0 & 0 & 0 & 0\\ 
                0 & 0 & 0 & -c/m_1 & 0 & 0 & 0 & 0\\
                0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\ 
                0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\ 
                0 & 0 & 0 & 0 & 0 & 0 & -c/m_2 & 0\\ 
                0 & 0 & 0 & 0 & 0 & 0 & 0 & -c/m_2\\ \end{bmatrix}
 \begin{bmatrix} x_1 \\ y_1 \\ \dot{x}_1 \\ \dot{y}_1 \\ x_2 \\ y_2 \\ \dot{x}_2 \\ \dot{y}_2\end{bmatrix} + 
 \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 1/m_1 & 0 \\ 0 & 1/m_1 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0\end{bmatrix}
 \begin{bmatrix} u_x^1 \\ u_y^1 \end{bmatrix} +
 \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 1/m_2 & 0 \\ 0 & 1/m_2\end{bmatrix}
 \begin{bmatrix} u_x^2 \\ u_y^2 \end{bmatrix}  $$


### Single Iteration of Affine QR

In [1]:
n = 4 #8 states
m = 2 #2 controls
function affinelq!(Aₜ, Bₜ, Qₜ, lₜ, Rₜ , rₜ)

    V = copy(Qₜ[:,:,end]) #at last time step
    ζ = copy(lₜ[:,end]) #at last time step
    #P = zeros(k_steps, n*m)
    P = zeros(Float32, (m, n, k_steps))
    #α = zeros(k_steps, m)
    α = zeros(Float32, (m, k_steps))

    #α₂ = zeros(T[1], m)

    for t in k_steps:-1:1
        # solving for Ps, check equation 19 in document
        # Player 1
        # Equation (10) only for 1 player for Pⱼ = 0 
        Pₜ = (Rₜ[:,:,t] + (Bₜ[:,:,t]' * V * Bₜ[:,:,t]))\(Bₜ[:,:,t]' * V * Aₜ[:,:,t])
        # Equation (11) only for 1 player for for αⱼ = 0
        αₜ = (Rₜ[:,:,t] + (Bₜ[:,:,t]' * V * Bₜ[:,:,t]))\(Bₜ[:,:,t]' * ζ)
        # Assign Values
        P[:,:,t] = Pₜ
        α[:,t] = αₜ
        #Update value function(s)
        # Equation (14)
        Fₜ = Aₜ[:,:,t] - (Bₜ[:,:,t]*Pₜ)
        # Equation (15)
        βₜ = - Bₜ[:,:,t] * αₜ
        # Equation (17)
        ζ = lₜ[:,t] + (Pₜ' * Rₜ[:,:,t] * αₜ - Pₜ' * rₜ[:,t]) + Fₜ'*(ζ + V * βₜ)
        # Equation (16) where V = Z
        V = Qₜ[:,:,t]' + (Pₜ' * Rₜ[:,:,t] * Pₜ) + (Fₜ' * V * Fₜ)
        #K[i, :] = reshape(transpose(K_i), (n*m,1))
        #println("f",reshape(K[i, :], (2,4))
    end
    return P, α
end

affinelq! (generic function with 1 method)

### Rollout a (nonlinear) trajectory

In [143]:
function Rollout_RK4(fun, x₀, xf, H, dt, P, α)
    """
    Rollout dynamics with initial state x₀ 
    and control law u = -Px - α
    P is an n x b gain matrix
    α is m x 1
    """
    m = 2 #2 controls
    k_steps = trunc(Int, H/dt) 
    xₜ = zeros(k_steps, length(x₀)) # 1500 x n
    uₜ = zeros(k_steps, m) 
    xₜ[1,:] .= x₀
    for t=1:(k_steps-1)
        # WHAT IS x̂ in xₜ[t,:] - x̂
        uₜ[t,:] = -P[:,:,t]*(xₜ[t,:] - xf) - α[:,t]
        k1 = fun(xₜ[t,:], uₜ[t,:])
        k2 = fun(xₜ[t,:] + 0.5*dt*k1, uₜ[t,:])
        k3 = fun(xₜ[t,:] + 0.5*dt*k2, uₜ[t,:])
        k4 = fun(xₜ[t,:] + dt*k3, uₜ[t,:])
        xₜ[t+1,:] = xₜ[t,:] + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
    end
    
    return xₜ, uₜ
end

Rollout_RK4 (generic function with 2 methods)

### Initiating P's and alpha's

In [144]:
x₀=[0; 0; 0; 0]
dt = 0.01 #step size
H = 30.0  #Horizon
P = zeros(2,4,3000)
α = zeros(2,3000)

2×3000 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [147]:
#function ilqr()
    # 1. Initialize an initial feasible trajectory
n = 4
m = 2
x₀ = [0; 0; 0; 0]
xf = [2; 2; 0; 0]
xₜ, uₜ = Rollout_RK4(point_mass, x₀, xf, H, dt, P, α) #xₜ is [k_steps, 4]
Aₜ = zeros(Float32, (n, n, k_steps))
Bₜ = zeros(Float32, (n, m, k_steps))
Qₜ = zeros(Float32, (n, n, k_steps))
lₜ = zeros(Float32, (n, k_steps))
Rₜ = zeros(Float32, (m, m, k_steps))
rₜ = zeros(Float32, (m, k_steps))

for t = 1:(k_steps)
    # 2. Linearize dynamics about trajectory
    # WHAT IS x̂ in xₜ[t,:] - x̂
    Aₜ[:,:,t], Bₜ[:,:,t] = lin_dyn_discrete(point_mass, xₜ[t,:] - xf, uₜ[t,:], dt)
    # 3. Compute second order Taylor series expansion the cost function
    # WHAT IS x̂ in xₜ[t,:] - x̂
    c, Qₜ[:,:,t], lₜ[:,t], Rₜ[:,:,t], rₜ[:,t] = quadratic_cost(cost, xₜ[t,:] - xf, uₜ[t,:])
end

# 4. Do lqr
P, α = affinelq!(Aₜ, Bₜ, Qₜ, lₜ, Rₜ , rₜ)

size(P)
#end

(2, 4, 3000)

In [148]:
xₜ, uₜ = Rollout_RK4(point_mass, x₀, xf, H, dt, P, α)

([0.0 0.0 0.0 0.0; 9.660896017670631e-5 9.660896017670631e-5 0.019130426862566472 0.019130426862566472; … ; 3.333526516700677 3.333526516700677 0.0004373745817753235 0.0004373745817753235; 3.3335307625407467 3.3335307625407467 0.0004120467906871093 0.0004120467906871093], [3.942016124725342 3.942016124725342; 3.9379452432799242 3.9379452432799242; … ; 2.9449865860996246e-5 2.9449865860996246e-5; 0.0 0.0])

In [150]:
xₜ[end,:]

4-element Vector{Float64}:
 3.3335307625407467
 3.3335307625407467
 0.0004120467906871093
 0.0004120467906871093